<a href="https://colab.research.google.com/github/edcalderin/BigDataEngineering_ICARO/blob/master/spark/practica_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
tar xf spark-2.4.7-bin-hadoop2.7.tgz
pip install -q findspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-2.4.7-bin-hadoop2.7'
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
quijote = spark.read.text('drive/MyDrive/Big_Data_Engineering/quijote.txt')
stop_words = spark.read.text('drive/MyDrive/Big_Data_Engineering/spanish_stop_words.txt')

#veamos que efectivamente es un DataFrame
print(type(quijote))
print(type(stop_words))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
print(quijote.printSchema())
print(stop_words.printSchema())

root
 |-- value: string (nullable = true)

None
root
 |-- value: string (nullable = true)

None


## Rdd

In [ ]:
quijote_rdd = quijote.rdd
stop_words_rdd = stop_words.rdd

In [12]:
import re

In [13]:
def normalize_text(line):
  regex = r'[^a-zA-Z ]'
  return re.sub(regex, '', line.lower().strip())

In [ ]:
quijote_rdd = quijote_rdd.map(lambda line: normalize_text(line[0]))

In [ ]:
quijote_rdd.take(5)

['el ingenioso hidalgo don quijote de la mancha',
 '',
 '',
 '',
 'por miguel de cervantes saavedra']

In [ ]:
quijote_word_list = quijote_rdd.flatMap(lambda line: line.split(' '))

In [ ]:
quijote_word_lens = quijote_word_list.map(lambda word: (word, len(word)))

In [ ]:
max_word_len = quijote_word_lens.reduce( lambda w1, w2: w1 if w1[1] > w2[1] else w2 )

In [ ]:
max_word_len

('bienintencionadamente', 21)

## Ejercicios

In [16]:
quijote = spark.read.text('drive/MyDrive/Big_Data_Engineering/quijote.txt').rdd

In [17]:
quijote = quijote.map(lambda line: normalize_text(line[0]))

In [31]:
%%time
len(quijote.reduce(lambda l1, l2: l1 if len(l1)>=len(l2) else l2))

CPU times: user 8.07 ms, sys: 147 µs, total: 8.21 ms
Wall time: 408 ms


75

In [40]:
map_quijote = quijote.map(lambda line: (line, len(line)))
map_quijote.takeOrdered(3, key=lambda x: -x[1])

[('donde le daba cuenta de su desgracia y de los amores del hijo del emperante',
  75),
 ('tan poco curiosos los ingenios de la mancha que no tuviesen en sus archivos',
  75),
 ('que en nuestra edad y en estos tan calamitosos tiempos se puso al trabajo y',
  75)]

In [43]:
key_value_quijote2 = quijote.map(lambda line: line.replace(' ','')).map(lambda line: (line, len(line)))
key_value_quijote2.takeOrdered(3, key=lambda x: -x[1])

[('intelegiblesdemonstrativosindubitablescondemostracionesmatemticas', 65),
 ('mientesenpartedondenoesposiblecorresponderosconformemerecevuestro', 65),
 ('aprovecharnosdelconocimientodiscretoparadistinguirestasdosmaneras', 65)]

## Palabra con menor frecuencia

In [49]:
quijote = spark.read.text('drive/MyDrive/Big_Data_Engineering/quijote.txt').rdd

In [85]:
lines_normalized = quijote.map(lambda line: normalize_text(line[0]))
lines_normalized = lines_normalized.filter(lambda line: line!='')\
                                    .flatMap(lambda line: line.split(' '))
lines_normalized = lines_normalized.map(lambda word: (word, 1))
sorted(lines_normalized.countByKey().items(), key=lambda x: x[1])[:3]

[('conste', 1), ('deciembre', 1), ('correcto', 1)]